# Avaliador sentimentos pelo CHAT GPT

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [8]:
import openai
import os
import pandas as pd
import time
import re
import os
import hashlib

openai.api_key = os.getenv('GPT_API_KEY')


def classifica_sentimento_noticia_gpt2(data, titulo, texto, empresa, model='gpt-4-turbo-preview', caminho_cache='datasets/gpt_cache.xlsx', usar_cache=True, dicionario_cache=None):
    '''Utiliza a API do Chat GPT para analisar uma noticia'''
    
    pergunta = '''Você receberá um texto de uma notícia sobre uma empresa, e deverá responder:'
                  a) se a notícia do tema ESG (considere que casos relacionados à inadimplência, falência ou recuperação judicial devem ser classificador na dimensão G); 
                  b) Em caso positivo, qual a dimensão dominante: E, S ou G; 
                  c) Avaliar o sentimento da notícia, considerando a atitude da empresa sob a ótica ESG, no intervalo real entre -1.0 até +1.0; 
                  d) Qual o nome da principal empresa envolvida; e) Esse texto tem como tema principal a empresa "<empresa>"? 
                  f) Se a notícia for do tema ESG, faça um breve resumo de apenas 1 sentença. Dê respostas curtas'''.replace('<empresa>', empresa)
    
    
    response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {
                      "role": "system",
                      "content": pergunta
                    },
                    {
                      "role": "user",
                      "content": titulo + '\n' + texto
                    }
                  ],
                temperature=0,
                max_tokens=255,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
                )


    resposta = response.choices[0].message["content"]


    return resposta


def trata_resposta_gpt(r):
    '''Rotina interna para tratar a resposta do chat gpt'''
    
    itens = ['a) ', 'b) ', 'c) ', 'd) ', 'e) ', 'f) ', 'g) ']
    
    r = r.replace('\n\n', '\n')
    
    respostas = r.split('\n')
    
    
    respostas = [resp.replace(item, '') for resp, item in zip(respostas, itens)]
    
    if len(respostas) <= 2:
        return 'Outros', 'Outros', '', '', '', ''
    
    try:
    
        tema = respostas[0][:3]

        if tema == 'Sim':
            tema = 'ESG'
        else:
            tema = 'Outros'

        dimensao = 'Outros'
        if 'S' in respostas[1] or 'G' in respostas[1] or 'E' in respostas[1]:
            dimensao = respostas[1].replace('A dimensão dominante é ', '').replace('"', '').strip()[:1]

        sentimento = 0
        nota = re.findall('[+-]?[0-9][.]?[0-9]{1,3}', respostas[2])
        if len(nota) > 0:
            sentimento = float(nota[0].strip())

        empresa_detectada = respostas[3].replace('A principal empresa envolvida é o ', '').replace('A principal empresa envolvida é a ', '').replace('.', '')

        foco_empresa = respostas[4][:3]
        
        if len(respostas) < 6:
            resumo = ''
        else:
            resumo = respostas[5]
            
    except Exception as e:
        logging.error(traceback.format_exc())
        print(respostas)
    
    return [tema, dimensao, sentimento, empresa_detectada, foco_empresa, resumo]


def gera_colunas_gpt(df, coluna_resposta_gpt='resposta'):
    '''rotina interna para fazer a separacao das respostas do chat gpt em colunas'''
    df['gpt_lista_respostas'] = df.apply(lambda row: trata_resposta_gpt(row[coluna_resposta_gpt]), axis=1)
    df['gpt_tema_esg'] = df['gpt_lista_respostas'].apply(lambda x : x[0])
    df['gpt_classificacao'] = df['gpt_lista_respostas'].apply(lambda x : x[1])
    df['gpt_polaridade'] = df['gpt_lista_respostas'].apply(lambda x : x[2])
    df['gpt_empresa_principal'] = df['gpt_lista_respostas'].apply(lambda x : x[3])
    df['gpt_focado_empresa'] = df['gpt_lista_respostas'].apply(lambda x : x[4])
    df['gpt_resumo'] = df['gpt_lista_respostas'].apply(lambda x : x[5])
    return df


In [11]:
texto = '''

Companhia incentiva o desenvolvimento socioeconômico no estado por meio do cultivo sustentável da palma de óleo

O Grupo BBF (Brasil BioFuels), produtor de óleo de palma na América Latina, participou no último dia 07/02, na sede da Agência de Defesa Agropecuária do Pará (Adepará), em Belém, da cerimônia de Apresentação da Guia de Trânsito Vegetal (GTV) do dendê, que busca combater o comércio ilegal de frutos de dendê no estado do Pará.

No encontro, Adepará, Faepa e Abrapalma, lideraram a agenda e apresentaram as ações da Portaria N°6143/2023, de 28 de dezembro de 2023, que instituiu a Política de Rastreabilidade para a Cadeia Produtiva da Palma de Óleo. A coalização para a rastreabilidade e combate ao comércio ilegal de dendê esclareceu dúvidas e recebeu contribuições para aprimoramento do processo de implementação da Guia de Trânsito Vegetal (GTV) do dendê, instrumento importante para o mecanismo de rastreio e fiscalização do transporte dos frutos, atestando a comprovação de origem e destino da produção. A portaria implementa ainda a obrigatoriedade do cadastramento das áreas de cultivo, unidades beneficiadoras e transportadores de cachos de frutos frescos de palma de óleo produzidos no estado. A norma tem prazo de 120 dias para começar a vigorar após a publicação que ocorreu em 02 de janeiro de 2024, portanto, a partir de 01 de maio de 2024.

O evento contou com as presenças da diretora de Defesa de Inspeção Vegetal da Adepará, Lucionila Pimentel; do vice-presidente da Abrapalma e diretor agrícola do Grupo BBF, Fábio Pacheco; do subdiretor de Organização Produtiva e Povos Tradicionais da Secretaria de Estado da Agricultura Familiar, Magnaldo Menezes; do representante da Fetagri, Ivaldo de Oliveira; do secretário-adjunto de Segurança Pública do estado do Pará, Luciano de Oliveira, entre outras autoridades. Representantes da Federação de Agricultura e Pecuária do Pará (FAEPA), secretários municipais de agricultura de todo o estado e representantes de associações de agricultores familiares também estiveram no encontro.

Para Lucionila Pimentel, da Adepará, a Guia de Trânsito Vegetal vai coibir irregularidades cometidas hoje na comercialização dos frutos de dendê no estado. “A recepção do produto só poderá acontecer mediante a guia de trânsito vegetal e o acompanhamento da guia de trânsito vegetal”, disse Lucionila. “Haverá um fiscal que vai acompanhar mensalmente a entrada, a recepção dos produtos e a organização das informações referentes à produção recebida. Então, o produto que entrar nas indústrias será acompanhado de uma intensa fiscalização”, completou a diretora.

Dendê - A cultura da palma de óleo, conhecida popularmente como dendê, dá origem ao óleo vegetal mais consumido no mundo e é muito importante para o desenvolvimento agrícola do estado. “Entendemos a necessidade da adoção de medidas que regulamentem a cadeia produtiva do dendê. Atualmente, o Grupo BBF é um dos principais produtores de palma no Pará, com mais de 60 mil hectares cultivados e incentivo de mais de 450 agricultores que fazem parte do Programa de Agricultura Familiar da empresa. Parabenizamos o esforço conjunto da coalizão e do governo do estado para que o Pará continue sendo referência na produção de palma de óleo no Brasil”, destaca Fábio Pacheco, diretor agrícola do Grupo BBF. De acordo com o Instituto Brasileiro de Geografia e Estatística (IBGE), o estado do Pará produz mais de 2,9 milhões de toneladas de dendê anualmente e lidera a produção nacional.

Já pela Abrapalma, o vice-presidente Fábio Pacheco, destaca que a Guia de Trânsito Vegetal será fundamental para que, inclusive a Segup, órgão de segurança pública do estado, possa atuar com a materialidade de eventuais ilícitos que podem constituir inclusive objeto para eventuais punições. “Isso que a gente espera para aqueles que cometem ilegalidades no comércio da palma. Hoje nós temos mais de duzentos mil hectares de plantios de palma vinculados à Abrapalma aqui no Pará, entre associados e produtores dos associados e que correm também um alto risco de problemas fitossanitários. Criar uma ferramenta tão poderosa como essa, para manter a fiscalização, conhecer a origem e destino dos frutos, ter um bom cadastro dos produtores, conhecer a condição dos plantios e os problemas fitossanitários que eventualmente tenham é fundamental para garantir o futuro e a sustentabilidade desse setor tão importante para o desenvolvimento da bioeconomia do estado do Pará”, conclui Pacheco.

Sobre o Grupo BBF

O Grupo BBF (Brasil BioFuels), empresa brasileira fundada em 2008, é produtor de óleo de palma na América Latina, com área cultivada superior a 75 mil hectares e capacidade de produção de cerca de 200 mil toneladas/ano de óleo. A empresa atua na criação de soluções sustentáveis para a geração de energia renovável nos sistemas isolados, com usinas termelétricas movidas a biocombustíveis produzidos na região. Sua atividade agrícola recupera áreas que foram degradadas até 2007 na Amazônia, seguindo o Zoneamento Agroecológico da Palma de Óleo (ZAE), aprovado pelo Decreto 7.172 do Governo Federal, de 7 de maio de 2010.

0 Grupo BBF criou um modelo de negócio integrado em que atua do início ao fim da cadeia de valor - desde o cultivo sustentável da palma de óleo, extração do óleo bruto, produção de bicombustíveis, biotecnologia e geração de energia renovável – com ativos totalizando cerca de R$ 2,2 bilhões e atividades gerando cerca de 6 mil empregos diretos na região Norte do Brasil. As operações do Grupo BBF estão situadas nos estados do Acre, Amazonas, Rondônia, Roraima e Pará, compreendendo 38 usinas termelétricas (25 em operação e 13 em implementação), 3 unidades de esmagamento de palma de óleo, uma extrusora de soja e uma indústria de biodiesel.

Website: http://www.grupobbf.com.br"
'''

In [12]:
classifica_sentimento_noticia_gpt2('','', texto, 'brasil biofuels')

'a) Sim, a notícia é do tema ESG.\n\nb) A dimensão dominante é E (Ambiental).\n\nc) O sentimento da notícia é positivo, avaliado em +0.8, pois destaca ações sustentáveis e de combate ao comércio ilegal.\n\nd) O nome da principal empresa envolvida é Grupo BBF (Brasil BioFuels).\n\ne) Sim, o texto tem como tema principal a empresa "Brasil BioFuels".\n\nf) O Grupo BBF participa de iniciativas para o cultivo sustentável da palma de óleo no Pará, promovendo a rastreabilidade e combatendo o comércio ilegal para garantir a sustentabilidade do setor.'